In [102]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [103]:
df = pd.read_csv('play_tennis.csv')
df

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


In [104]:
# First training is done by naive bayes
# In training phase it finds individual probablity for each type of event and store it in a lookup table
# Example like this:
dfOut =  pd.crosstab(df['outlook'], df['play'])
dfOut

play,No,Yes
outlook,,
Overcast,0,4
Rain,2,3
Sunny,3,2


In [105]:
dfOut['No'] = dfOut['No'].divide(dfOut['No'].sum()) 
dfOut['Yes'] = dfOut['Yes'].divide(dfOut['Yes'].sum()) 

In [106]:
## This is the probablity martrix
# Where this shows the probablity of playing or not playing under that condition
dfOut

play,No,Yes
outlook,,
Overcast,0.0,0.444444
Rain,0.4,0.333333
Sunny,0.6,0.222222


In [107]:
df = df.iloc[:,1:]
df

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [108]:
OneHotCol = df.iloc[:,:-1].columns
OneHotCol = OneHotCol.to_list()

In [109]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first',dtype=np.int32)

In [110]:
X_new = ohe.fit_transform(df[OneHotCol]).toarray()
newMat = pd.DataFrame(X_new,columns=ohe.get_feature_names_out(OneHotCol))
newMat

,outlook_Rain,outlook_Sunny,temp_Hot,temp_Mild,humidity_Normal,wind_Weak
0,0,1,1,0,0,1
1,0,1,1,0,0,0
2,0,0,1,0,0,1
3,1,0,0,1,0,1
4,1,0,0,0,1,1
5,1,0,0,0,1,0
6,0,0,0,0,1,0
7,0,1,0,1,0,1
8,0,1,0,0,1,1
9,1,0,0,1,1,1


In [111]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = df.iloc[:,-1]

In [112]:
newTarget = pd.Series(le.fit_transform(y), name = 'Play')
newMat['Play'] = newTarget

In [113]:
df = newMat

In [114]:
df

,outlook_Rain,outlook_Sunny,temp_Hot,temp_Mild,humidity_Normal,wind_Weak,Play
0,0,1,1,0,0,1,0
1,0,1,1,0,0,0,0
2,0,0,1,0,0,1,1
3,1,0,0,1,0,1,1
4,1,0,0,0,1,1,1
5,1,0,0,0,1,0,0
6,0,0,0,0,1,0,1
7,0,1,0,1,0,1,0
8,0,1,0,0,1,1,1
9,1,0,0,1,1,1,1


In [115]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [116]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state= 42, shuffle= True)

In [117]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(x_train, y_train)

GaussianNB()

In [118]:
y_Pred = classifier.predict(x_test)

In [119]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_Pred)
print(cm)
accuracy_score(y_test, y_Pred)

[[1 1]
 [1 2]]


0.6